# Output Parsers Implementation

### Imports

In [16]:
import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [5]:
# Get the OpenAI Key
OpenAI.openai_api_key = os.getenv("OPENAI_API_KEY")

### Comma Separated List

In [8]:
# We will get the format instruction - which is predefined and we pass the same to the LLM
output_parser = CommaSeparatedListOutputParser()

In [9]:
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [11]:
# Now we will build the template
prompt = PromptTemplate(
    template = "Provide 5 examples of {query}.\n{format_instructions}",
    input_variables = {"query"},
    partial_variables = {"format_instructions": format_instructions}
)

In [14]:
# build the prompt
prompt = prompt.format(query="Currencies")
print(prompt)

Provide 5 examples of Currencies.
Your response should be a list of comma separated values, eg: `foo, bar, baz`


In [15]:
# lets pass this prompt to the llm
llm = OpenAI(temperature=0.9, model_name="text-davinci-003")
output = llm(prompt)
print(output)



US Dollar, Euro, Pound Sterling, Japanese Yen, Canadian Dollar


### Json Format

In [17]:
# First we define the response schema
response_schemas = [
    ResponseSchema(name="currency", description="answer to the user's question"),
    ResponseSchema(name="abbreviation", description="Whats the abbrevation of that currency")
]

In [18]:
# initialize output parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
print(output_parser)

response_schemas=[ResponseSchema(name='currency', description="answer to the user's question", type='string'), ResponseSchema(name='abbreviation', description='Whats the abbrevation of that currency', type='string')]


In [19]:
# Now we will derive the format instructions from this output parser
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"currency": string  // answer to the user's question
	"abbreviation": string  // Whats the abbrevation of that currency
}
```


In [22]:
# Now we build the prompt template
prompt = PromptTemplate(
    template = "answer the users question as best as possible.\n {format_instructions}\n{query}",
    input_variables = {"query"},
    partial_variables = {"format_instructions": format_instructions}
)

In [23]:
# build the prompt
prompt = prompt.format(query="What is the currency of India?")
print(prompt)

answer the users question as best as possible.
 The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"currency": string  // answer to the user's question
	"abbreviation": string  // Whats the abbrevation of that currency
}
```
What is the currency of India?


In [24]:
# Now we will pass this to the llm
output = llm(prompt)
print(output)



```json
{
	"currency": "Indian Rupee" 
	"abbreviation": "INR" 
}
```
